In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')
project_dir = os.path.join('drive','My Drive','Personal Projects','Sleep Apnea Study','apnea-study')
os.chdir(project_dir)
!git pull
!pip install -r colab_requirements.txt 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Already up to date.
     |████████████████████████████████| 1.1MB 2.9MB/s 
     |████████████████████████████████| 380.5MB 60kB/s 
     |████████████████████████████████| 4.3MB 30.8MB/s 
     |████████████████████████████████| 501kB 40.5MB/s 
  Created wheel for pyEDFlib: filename=pyEDFlib-0.1.14-cp36-cp36m-linux_x86_64.whl size=920344 sha256=a077405c54ea8486e17efe40bbaeb1699fbc1c3044db7f16c3953b004bfb34ce
  Stored in directory: /root/.cache/pip/wheels/6a/62/a

### Imports

In [0]:


import datetime

import tensorflow as tf

import lib.utils as utils
from lib.models import BaseRNN, BaseLSTM, DoubleLSTM
from lib.train import train, evaluate

### Load Raw Data

In [0]:
#create_tfrecords_from_raw_data(raw_data_dir='data/raw_data',tf_rec_data_dir='data/processed_data')

### Inputs and Parameters

In [0]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
session_name = 'small_stride_test' +  '_' + current_time
optimizer_params = {'learning_rate':0.0001}
training_params = {'num_epochs':200, 'batch_size':256,'apnea_weight':5}
preprocess_params = {'featurize_func' : utils.featurize_2,'seq_len':60,'pulse_sample_rate':16,'data_stride':5}
preprocess_func = utils.preprocess_data
model_params = {'rnn_hidden_dim':20}
test_split = .15
val_split = 0.1765


with tf.device("gpu:0"):
    model = BaseLSTM(rnn_hidden_dim=model_params['rnn_hidden_dim'])
    loss_object = tf.keras.losses.BinaryCrossentropy()
    optimizer = tf.keras.optimizers.Adam(optimizer_params['learning_rate'])


log_dir = 'logs/gradient_tape/' + session_name
model_weights_dir = 'model_weights/' + session_name

In [0]:
dataset = utils.load_tfrecords(tf_rec_data_dir='data/processed_data')


a = preprocess_func(dataset.take(2), 
                                 featurize_func = preprocess_params['featurize_func'],
                                 seq_len=preprocess_params['seq_len'], 
                                 pulse_sample_rate=preprocess_params['pulse_sample_rate'],
                                 data_stride=preprocess_params['data_stride'])

In [0]:
a._tensors[0].shape[0] / 3600

1.6966666666666668

### Preprocessing and Training

In [0]:

dataset = utils.load_tfrecords(tf_rec_data_dir='data/processed_data')
num_records = 0
for i in dataset:
    num_records += 1

train_data, test_data = utils.split_dataset(dataset, test_split)
train_data, val_data = utils.split_dataset(train_data, val_split)
with tf.device("gpu:0"):

    train_data = preprocess_func(train_data, 
                                 featurize_func = preprocess_params['featurize_func'],
                                 seq_len=preprocess_params['seq_len'], 
                                 pulse_sample_rate=preprocess_params['pulse_sample_rate'],
                                 data_stride=preprocess_params['data_stride'])
    val_data = preprocess_func(val_data, 
                               featurize_func = preprocess_params['featurize_func'],
                               seq_len=preprocess_params['seq_len'], 
                               pulse_sample_rate=preprocess_params['pulse_sample_rate'],
                               data_stride=preprocess_params['seq_len'])
    test_data = preprocess_func(test_data, 
                                featurize_func = preprocess_params['featurize_func'],
                                seq_len=preprocess_params['seq_len'], 
                                pulse_sample_rate=preprocess_params['pulse_sample_rate'],
                                data_stride=preprocess_params['seq_len'])

train_num = train_data._tensors[0].shape[0]
train_bal = train_data._tensors[1].numpy().mean()
val_num = val_data._tensors[0].shape[0]
val_bal = val_data._tensors[1].numpy().mean()
test_num = test_data._tensors[0].shape[0]
test_bal = test_data._tensors[1].numpy().mean()

data_bal = {'train':train_bal,'val':val_bal,'test':test_bal}
data_size = {'train':train_num,'val':val_num,'test':test_num}
print(train_num)
print(train_bal)
print(val_num)
print(val_bal)
print(test_num)
print(test_bal)

W0902 15:35:26.737979 139819408115584 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/data/util/random_seed.py:58: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


164290
0.09518463
3054
0.09882668
3170
0.053222924


In [0]:
%load_ext tensorboard
print(tf.test.is_gpu_available())
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
True


Reusing TensorBoard on port 6006 (pid 3267), started 0:00:07 ago. (Use '!kill 3267' to kill it.)

In [0]:
with tf.device("gpu:0"):

    train(model, train_data,val_data, loss_object, optimizer, log_dir,model_weights_dir,
          num_epochs=training_params['num_epochs'], 
          batch_size=training_params['batch_size'],
          apnea_weight=training_params['apnea_weight'])
train_res = evaluate(model, train_data)
test_res = evaluate(model, test_data)
print(train_res)
print(test_res)
utils.save_session(session_name,
                   model,
                   model_params,
                   num_records,
                   test_split,
                   val_split,
                   preprocess_func,
                   preprocess_params,
                   data_bal,
                   data_size,
                   training_params,
                   optimizer,
                   optimizer_params,
                   train_res,
                   test_res,
                   log_dir,
                   model_weights_dir,
                   res_path = 'results')

Starting Training


KeyboardInterrupt: ignored

In [0]:
evaluate(model, val_data)

### Evaluation

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from lib.utils import to_dense_tensors
%matplotlib inline

dataset = load_tfrecords(tf_rec_data_dir='data/processed_data')
dataset = dataset.map(to_dense_tensors)
for dp in dataset:
    pass
x = dp['x']
y = dp['y']
plt.plot((x - tf.math.reduce_mean(x)) / tf.math.reduce_std(x))
plt.plot(np.repeat(y,16))


In [0]:
pulse_sample_rate = 16
data_stride = 30
seq_len = 16
X = []
Y = []
#normalize pulse data by night of sleep
x = (x - tf.math.reduce_mean(x)) / tf.math.reduce_std(x)
#convert night of sleep from 1 dimensional sequence of num samples length
#to a sequence of pulse_sample_rate dimension of num seconds length
num_seconds = x.shape[0]//pulse_sample_rate
x_trunc = x[:pulse_sample_rate*(num_seconds)]
x = tf.reshape(x_trunc,[num_seconds,pulse_sample_rate])
#create new datapoints, according to data_stride
num_data_points = (x.shape[0]//data_stride) - (seq_len//data_stride)
for i in range(0,num_data_points):
    X.append(x[data_stride*i:data_stride*i+seq_len])
    Y.append(y[data_stride*i:data_stride*i+seq_len])

In [0]:
(np.absolute(x[data_stride*i:data_stride*i+seq_len].numpy().flatten()) > 1.5).any()

In [0]:
type(x[data_stride*i:data_stride*i+seq_len])

In [0]:
y[data_stride*i:data_stride*i+seq_len].numpy().mean() == 0

### ToDo and Scrap

In [0]:
#fix learning rate
#visualize function
#develop model search procedure - preprocessing, architecture, training params
#set seeds

#remove outliers
#0-1 scaling 


#write main.py with argparse
#write shell script for parameter tuning
#set up interpretion, eda notebook
#turn model params into keyword args
#add metrics to tensorboard
#add keyboard interupt to training loop

import pandas as pd
import os
import re
import pickle
os.getcwd()
results_path = 'results'

def get_resuts_df(results_path):
    """Summarizes session result pickles in a dataframe
    
    Parameters:
    ------------
    
    results_path (str) : path to session result pickles
    
    Returns:
    ---------
    results_df (pd.DataFrame) : results in dataframe format
    
    
    """
    results_exp = re.compile(r'.*\.pickle$')
    results_list = os.listdir(results_path)
    results_list = list(filter(results_exp.search,results_list))
    results_df = []
    for result_file in results_list:
        results_full_path = os.path.join(results_path,result_file)
        with open(results_full_path,'rb') as file:
            session_results = pickle.load(file)
            session_series = pd.Series()
            session_series['session_name'] = result_file
            for key in session_results:
                if isinstance(session_results[key],dict):
                    for subkey in session_results[key]:
                        session_series[key+'_'+subkey] = session_results[key][subkey]
                else:
                    session_series[key] = session_results[key]
            results_df.append(session_series)
    results_df = pd.DataFrame(results_df)
    return results_df

df = get_resuts_df(results_path)
session_cols = ['session_name','data_bal_train','data_bal_test']
result_cols = ['train_res_recall','train_res_precision','train_res_f1',	'train_res_auc',
               'test_res_accuracy', 'test_res_recall', 'test_res_precision', 'test_res_f1',	'test_res_auc']

df[session_cols+result_cols]
